In [9]:
#get processed data

import pandas as pd

lut = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/lutein.csv?raw=true")

sug = pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/sugars.csv?raw=true")

wat=pd.read_csv("https://github.com/Chike-Ode/Imputation_Research_Project/blob/master/data/processed/water.csv?raw=true")

#dropping Name column
lut=lut.drop('name',axis=1)
sug=sug.drop('name',axis=1)
wat=wat.drop('name',axis=1)

#adding indicators to retrieve rows that were NaN after imputation
lut['mcarna'] = np.where(np.isnan(lut['lutein_zeaxanthin_mcar'].values), 1, 0) 
lut['marna'] = np.where(np.isnan(lut['lutein_zeaxanthin_mar'].values), 1, 0) 

sug['mcarna'] = np.where(np.isnan(sug['sugars_mcar'].values), 1, 0) 
sug['marna'] = np.where(np.isnan(sug['sugars_mar'].values), 1, 0) 

wat['mcarna'] = np.where(np.isnan(wat['water_mcar'].values), 1, 0) 
wat['marna'] = np.where(np.isnan(wat['water_mar'].values), 1, 0)



In [3]:
pip install MissForestExtra

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install missingpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-learn==0.22.1

  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
    Running setup.py install for scikit-learn: started
    Running setup.py install for scikit-learn: finished with status 'error'
  Rolling back uninstall of scikit-learn
  Moving to c:\users\gabri\anaconda3\lib\site-packages\scikit_learn-0.24.2.dist-info\
   from C:\Users\gabri\anaconda3\Lib\site-packages\~cikit_learn-0.24.2.dist-info
  Moving to c:\users\gabri\anaconda3\lib\site-packages\sklearn\
   from C:\Users\gabri\anaconda3\Lib\site-packages\~klearn
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\gabri\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\gabri\\AppData\\Local\\Temp\\pip-install-8591uz45\\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\\setup.py'"'"'; __file__='"'"'C:\\Users\\gabri\\AppData\\Local\\Temp\\pip-install-8591uz45\\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\gabri\AppData\Local\Temp\pip-wheel-exxdqgex'
       cwd: C:\Users\gabri\AppData\Local\Temp\pip-install-8591uz45\scikit-learn_34882d4bceb7493ab8dc61a9564241f3\
  Complete output (35 lines):
  Partial import of sklearn during the build process.
  No module named 'numpy.distutils._msvccompi

In [19]:
#create arrays with only the rows that have NaN to measure performance on at the end
import numpy as np

#lutmarnan2=lut[np.isnan(lut['lutein_zeaxanthin_mar'])]
lutmarnan=lut.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)

#lutmcarnan2=lut[np.isnan(lut['lutein_zeaxanthin_mcar'])]
lutmcarnan=lut.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)

#sugmarnan2=sug[np.isnan(sug['sugars_mar'])]
sugmarnan=sug.drop(['sugars_mcar','sugars'],axis=1)

#sugmcarnan2=sug[np.isnan(sug['sugars_mcar'])]
sugmcarnan=sug.drop(['sugars_mar','sugars'],axis=1)

#watmarnan2=wat[np.isnan(wat['water_mar'])]
watmarnan=wat.drop(['water_mcar','water'],axis=1)

#watmcarnan2=wat[np.isnan(wat['water_mcar'])]
watmcarnan=wat.drop(['water_mar','water'],axis=1)

lutmarnan=lutmarnan.to_numpy()
lutmcarnan=lutmcarnan.to_numpy()

sugmarnan=sugmarnan.to_numpy()
sugmcarnan=sugmcarnan.to_numpy()

watmarnan=watmarnan.to_numpy()
watmcarnan=watmcarnan.to_numpy()

In [20]:
#replace NaN using MICE algorithm
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from missingpy import MissForest

# Make an instance and perform the imputation
imp = MissForest()

#lutein mcar
#Need to drop lutein mar and true lutein columns
lutmcar=lut.copy()
lutmcar = lutmcar[lutmcar['lutein_zeaxanthin_mcar'].notnull()]
lutmcar=lutmcar.drop(['lutein_zeaxanthin_mar','lutein_zeaxanthin'],axis=1)
lutmcarn=lutmcar.to_numpy()
imp.fit(lutmcarn)
lutmcar2=imp.transform(lutmcarnan)

lutmcarimp = pd.DataFrame(lutmcar2, columns = lutmcar.columns)

#lutein mar
#Need to drop lutein mcar and true lutein columns
lutmar=lut.copy()
lutmar = lutmar[lutmar['lutein_zeaxanthin_mar'].notnull()]
lutmar=lutmar.drop(['lutein_zeaxanthin_mcar','lutein_zeaxanthin'],axis=1)
lutmarn=lutmar.to_numpy()
imp.fit(lutmarn)
lutmar2=imp.transform(lutmarnan)

lutmarimp = pd.DataFrame(lutmar2, columns = lutmar.columns)

#sugars mcar
#Need to drop sugar mar and true sugar columns
sugmcar=sug.copy()
sugmcar = sugmcar[sugmcar['sugars_mcar'].notnull()]
sugmcar=sugmcar.drop(['sugars_mar','sugars'],axis=1)
sugmcarn=sugmcar.to_numpy()
imp.fit(sugmcarn)
sugmcar2=imp.transform(sugmcarnan)
sugmcarimp = pd.DataFrame(sugmcar2, columns = sugmcar.columns)


#sugars mar
#Need to drop sugar mcar and true sugar columns
sugmar=sug.copy()
sugmar = sugmar[sugmar['sugars_mar'].notnull()]
sugmar=sugmar.drop(['sugars_mcar','sugars'],axis=1)
sugmarn=sugmar.to_numpy()
imp.fit(sugmarn)
sugmar2=imp.transform(sugmarnan)
sugmarimp = pd.DataFrame(sugmar2, columns = sugmar.columns)


#water mcar
#Need to drop water mar and true water columns
watmcar=wat.copy()
watmcar = watmcar[watmcar['water_mcar'].notnull()]
watmcar=watmcar.drop(['water_mar','water'],axis=1)
watmcarn=watmcar.to_numpy()
imp.fit(watmcarn)
watmcar2=imp.transform(watmcarnan)
watmcarimp = pd.DataFrame(watmcar2, columns = watmcar.columns)


#water mar
#Need to drop water mcar and true water columns
watmar=wat.copy()
watmar = watmar[watmar['water_mar'].notnull()]
watmar=watmar.drop(['water_mcar','water'],axis=1)
watmarn=watmar.to_numpy()
imp.fit(watmarn)
watmar2=imp.transform(watmarnan)
watmarimp = pd.DataFrame(watmar2, columns = watmar.columns)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3


In [18]:
print(lutmcarnan)

[[ 2.4  0.4  0.  ...  nan  1.   1. ]
 [ 2.   1.2  1.  ...  nan  1.   1. ]
 [ 0.3  0.1  0.  ...  nan  1.   1. ]
 ...
 [ 6.8  2.3 77.  ...  nan  1.   0. ]
 [ 5.4  1.8 63.  ...  nan  1.   1. ]
 [29.  12.  82.  ...  nan  1.   1. ]]


In [23]:
#adding back the column with true values

lutmcarimp['true_lut'] = lut['lutein_zeaxanthin'].values
lutmcarimp['mcarna'] = lut['mcarna'].values

lutmarimp['true_lut'] = lut['lutein_zeaxanthin'].values
lutmarimp['marna'] = lut['marna'].values

sugmcarimp['true_sug'] = sug['sugars'].values
sugmcarimp['mcarna'] = sug['mcarna'].values

sugmarimp['true_sug'] = sug['sugars'].values
sugmcarimp['marna'] = sug['marna'].values

watmcarimp['true_wat'] = wat['water'].values
watmcarimp['mcarna'] = wat['mcarna'].values

watmarimp['true_wat'] = wat['water'].values
watmarimp['marna'] = wat['marna'].values

#filter on previously missing values

lutmcarimp=lutmcarimp[lutmcarimp['mcarna'] == 1]
lutmarimp=lutmarimp[lutmarimp['marna'] == 1]

sugmcarimp=sugmcarimp[sugmcarimp['mcarna'] == 1]
sugmarimp=sugmarimp[sugmarimp['marna'] == 1]

watmcarimp=watmcarimp[watmcarimp['mcarna'] == 1]
watmarimp=watmarimp[watmarimp['marna'] == 1]


In [24]:
#calculating rmse for each columns and missing type
import math
import numpy as np


meth='RF'
results=[]

#lutein mcar
dat='lutein'
dt=lutmcarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mcar'])**2
lutmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmcarrmse)
results.append([dat,'mcar',meth,lutmcarrmse])

#lutein mar
dt=lutmarimp

dt['sqrt']=(dt['true_lut']-dt['lutein_zeaxanthin_mar'])**2
lutmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",lutmarrmse)
results.append([dat,'mar',meth,lutmarrmse])

#sugars mcar
dat='sugars'
dt=sugmcarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mcar'])**2
sugmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmcarrmse)
results.append([dat,'mcar',meth,sugmcarrmse])

#sugars mar
dt=sugmarimp

dt['sqrt']=(dt['true_sug']-dt['sugars_mar'])**2
sugmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",sugmarrmse)
results.append([dat,'mar',meth,sugmarrmse])

#water mcar
dat='water'
dt=watmcarimp

dt['sqrt']=(dt['true_wat']-dt['water_mcar'])**2
watmcarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmcarrmse)
results.append([dat,'mcar',meth,watmcarrmse])

#water mcar
dt=watmarimp

dt['sqrt']=(dt['true_wat']-dt['water_mar'])**2
watmarrmse=math.sqrt(dt['sqrt'].mean())
print("RMSE:",watmarrmse)
results.append([dat,'mar',meth,watmarrmse])

print(results)

RMSE: 671.6238584451428
RMSE: 850.1817194931109
RMSE: 6.159290249197037
RMSE: 4.714843521156505
RMSE: 3.3438381853984382
RMSE: 5.466549827500337
[['lutein', 'mcar', 'RF', 671.6238584451428], ['lutein', 'mar', 'RF', 850.1817194931109], ['sugars', 'mcar', 'RF', 6.159290249197037], ['sugars', 'mar', 'RF', 4.714843521156505], ['water', 'mcar', 'RF', 3.3438381853984382], ['water', 'mar', 'RF', 5.466549827500337]]


In [25]:
res = pd.DataFrame(results, columns = ['Column', 'Missing type','Imputation method','RMSE'])
print(res)

   Column Missing type Imputation method        RMSE
0  lutein         mcar                RF  671.623858
1  lutein          mar                RF  850.181719
2  sugars         mcar                RF    6.159290
3  sugars          mar                RF    4.714844
4   water         mcar                RF    3.343838
5   water          mar                RF    5.466550
